In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Sequential, Model 

<H1>Using custom CNN architecture to train the cancer detection model</H1>

In [15]:
classifier = Sequential()
classifier.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(128, 128, 3)))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Dropout(0.2))
classifier.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Dropout(0.2))
classifier.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Dropout(0.2))
classifier.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Dropout(0.1))
classifier.add(GlobalAveragePooling2D())
classifier.add(Dense(3, activation='softmax'))
classifier.summary()
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 64, 64, 64)        9280      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 128)       73856     
__________

In [16]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory("dataset/train",target_size = (128, 128),batch_size = 50,class_mode = 'categorical')
test_set = test_datagen.flow_from_directory("dataset/valid",target_size = (128, 128),batch_size = 50,class_mode = 'categorical')

Found 2002 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model_without_transfer_learning.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
classifier.fit_generator(training_set, epochs = 100, validation_data = test_set, callbacks = [checkpoint], shuffle=True)

Epoch 1/100
41/41 [==============================] - 871s 21s/step - loss: 0.8550 - acc: 0.6935 - val_loss: 1.0435 - val_acc: 0.5200

Epoch 00001: val_acc improved from -inf to 0.52000, saving model to model_without_transfer_learning.h5
Epoch 2/100
32/41 [======================>.......] - ETA: 3:06 - loss: 0.7988 - acc: 0.7069

<H1>Prediction from the trained model</H1>

In [19]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
test_image = image.load_img('dataset/train/melanoma/ISIC_0015045.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
model = load_model('model_without_transfer_learning.h5')
result = classifier.predict(test_image)
print(result)

[[0. 1. 0.]]


<H1>Using transfer learning to train the cancer detection model</H1>

In [ ]:
from keras import applications
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))
print(model.summary())

In [ ]:
for layer in model.layers[:5]:
    layer.trainable = False
x = model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(100, activation="relu")(x)
x = Dense(100, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)
model_final = Model(input = model.input, output = predictions)

In [ ]:
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
path = "dataset/train"
path_test = "dataset/valid"
training_set = train_datagen.flow_from_directory(path,target_size = (128, 128),batch_size = 40, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(path_test,target_size = (128, 128),batch_size = 40, class_mode = 'categorical')

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model_using_transfer_learning.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
classifier.fit_generator(training_set, epochs = 10, validation_data = test_set, callbacks = [checkpoint], shuffle=True)

In [ ]:
model_final.summary()